In [49]:
import os
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [50]:
path_to_data = r'C:\Users\Rory\raw_data\SERT_DREADD\spikes_df'
recordings_to_analyse = ['2018-04-18']

In [51]:
def zscore(group):
    return (group['mean_firing_rate'].subtract(baseline_means)).divide(baseline_stds)

In [52]:
file = '\\'.join([path_to_data, recordings_to_analyse[0]]) + '.csv'
df = pd.read_csv(file, index_col=0)
df['spike'] = 1

In [53]:
# compute mean in first period

baseline = df.loc[df['condition'] == 'Baseline']
mean_firing_rates = baseline.groupby('spike_cluster')['time'].agg(lambda x: x.count() /float(x.max()-x.min()))

In [54]:
df['time'] = pd.to_timedelta(df['time'], unit='s')

In [55]:
baseline = df.loc[df['condition'] == 'Baseline']
base_ts = df.pivot_table(index='time', columns='spike_cluster', values='spike', aggfunc='count')

In [56]:
base_sec = base_ts.resample('s').count()

In [60]:
baseline_means = base_sec.transpose().mean(axis=1)
baseline_stds = base_sec.transpose().std(axis=1)
baseline_sorted = baseline_means.sort_values()
baseline_sorted.head()

spike_cluster
401    0.498016
215    0.556726
159    0.655809
219    0.761872
92     0.837964
dtype: float64

In [61]:
all_data_ts = df.pivot_table(index='time', columns='spike_cluster', values='spike', aggfunc='count')
secondly = all_data_ts.resample('s').count()
all_data_rolling_mean = secondly.rolling(60*2).mean()

In [62]:
tidy_ts = all_data_ts.reset_index().melt(id_vars='time', value_name='mean_firing_rate').set_index('spike_cluster')

In [63]:
#works but slow!!!!
normed = tidy_ts.groupby('time').apply(zscore)

In [ ]:
f, a = plt.subplots(figsize=(19, 9))
sns.heatmap(normed.transpose(), ax=a, cmap='coolwarm', vmin=-3, vmax=3, xticklabels=1800)
a.set_ylabel('Neuron Number')
a.set_title('Percentage Baseline firing rate\nTwo minute rolling window')
a.set_xticklabels('30 60 120 150'.split())
a.set_xlabel('Time (min)\nCNO administration at 120 minutes')

Text(0.5,61,'Time (min)\nCNO administration at 120 minutes')